In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

import torchvision.models as models
import torchvision.datasets as dset
import torchvision.transforms as transforms

import os
import cv2
import copy
import time
from random import *
from collections import defaultdict

from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('train.csv')
test  = pd.read_csv('test.csv')
submission = pd.read_csv('submission.csv')

In [3]:
train.head()

,id,digit,letter,0,1,2,3,4,5,6,...,774,775,776,777,778,779,780,781,782,783
0,1,5,L,1,1,1,4,3,0,0,...,2,1,0,1,2,4,4,4,3,4
1,2,0,B,0,4,0,0,4,1,1,...,0,3,0,1,4,1,4,2,1,2
2,3,4,L,1,1,2,2,1,1,1,...,3,3,3,0,2,0,3,0,2,2
3,4,9,D,1,2,0,2,0,4,0,...,3,3,2,0,1,4,0,0,1,1
4,5,6,A,3,0,2,4,0,3,0,...,4,4,3,2,1,3,4,3,1,2


In [4]:
x_train = np.concatenate(
    [
        pd.get_dummies(train['letter']).values.reshape(-1, 1, 26),
        (train[[str(i) for i in range(784)]] / 255.).values.reshape(-1, 1, 784)
    ],
    axis=2
)
y_train = train['digit'].values

In [5]:
x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, test_size=0.2, random_state=123)

In [6]:
print(x_train.shape)
print(x_valid.shape)
print(y_train.shape)
print(y_valid.shape)

(1638, 1, 810)
(410, 1, 810)
(1638,)
(410,)


In [7]:
# tensor로 형변환
x_train = torch.Tensor(x_train)
x_valid = torch.Tensor(x_valid)
y_train = torch.LongTensor(y_train)
y_valid = torch.LongTensor(y_valid)

In [8]:
train_data = TensorDataset(
    x_train[:, :, :26], # Letter
    x_train[:, :, 26:].reshape(-1, 1, 28, 28), # Image
    y_train # Digit
)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=32)

valid_data = TensorDataset(
    x_valid[:, :, :26],
    x_valid[:, :, 26:].reshape(-1, 1, 28, 28),
    y_valid
)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=32)

In [9]:
class customCNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Letter의 Convolution Block
        self.conv1 = nn.Sequential(
            nn.Conv1d(1, 16, 3, padding=1), nn.ReLU(), # 16@26
            nn.Conv1d(16, 64, 4, padding=1), nn.ReLU(), # 64@25
            nn.Conv1d(64, 128, 5, padding=2), nn.ReLU(), # 128@25
            nn.Conv1d(128, 64, 4, padding=2), nn.ReLU(), # 64@26
            nn.Conv1d(64, 16, 3), nn.ReLU(), # 16@24
        )
        
        # Image의 Convolution Block
        self.conv2 = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(), # 32@28x28
            nn.Conv2d(32, 128, 5, padding=2), nn.ReLU(), # 128@28x28
            nn.Conv2d(128, 256, 7, padding=3), nn.ReLU(), # 256@28x28
            nn.Conv2d(256, 512, 9, padding=3), nn.ReLU(), # 512@26x26
            nn.Conv2d(512, 256, 9, padding=3), nn.ReLU(), # 256@24x24
            nn.Conv2d(256, 128, 7, padding=3), nn.ReLU(), # 128@24x24
            nn.Conv2d(128, 64, 7, padding=3), nn.ReLU(), # 64@24x24
            nn.Conv2d(64, 32, 5, padding=3), nn.ReLU(), # 32@26x26
        )
        
        '''
            Output Size = (W + 2xP - Dx(K-1) -1) / S + 1

            W: input_volume_size
            K: kernel_size
            D: Dilation
            P: padding_size
            S: strides
        '''
        
        self.out = nn.Sequential(
            nn.Linear(22016, 128), nn.ReLU(),
            nn.Linear(128, 32), nn.ReLU(),
            nn.Linear(32, 10)
        )
        
        self.loss = nn.CrossEntropyLoss()
    
    def forward(self, x1, x2, label=False):
        out = self._inference(x1, x2)
        if label is not False:
            loss = self.loss(out, label)
            return (out, loss)
        
        return out
    
    def _inference(self, x1, x2):
        bsz = x1.size(0)
#         print('bsz: ', bsz)
        x1 = self.conv1(x1)
#         print('x1 shape: ', x1.shape) # [32, 16, 24]
        x2 = self.conv2(x2)
#         print('x2 shape: ', x2.shape) # [32, 32, 26, 26]
        
        x1 = x1.view(bsz, -1)
#         print('x1 shape: ', x1.shape) # [32, 384]
        x2 = x2.view(bsz, -1)
#         print('x2 shape: ', x2.shape) # [32, 21632]
        
        x = torch.cat([x1, x2], dim=1)
        out = torch.nn.functional.softmax(self.out(x), dim=1)
        
        return out

In [10]:
model = customCNN()
model.cuda()

customCNN(
  (conv1): Sequential(
    (0): Conv1d(1, 16, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): Conv1d(16, 64, kernel_size=(4,), stride=(1,), padding=(1,))
    (3): ReLU()
    (4): Conv1d(64, 128, kernel_size=(5,), stride=(1,), padding=(2,))
    (5): ReLU()
    (6): Conv1d(128, 64, kernel_size=(4,), stride=(1,), padding=(2,))
    (7): ReLU()
    (8): Conv1d(64, 16, kernel_size=(3,), stride=(1,))
    (9): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): ReLU()
    (4): Conv2d(128, 256, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (5): ReLU()
    (6): Conv2d(256, 512, kernel_size=(9, 9), stride=(1, 1), padding=(3, 3))
    (7): ReLU()
    (8): Conv2d(512, 256, kernel_size=(9, 9), stride=(1, 1), padding=(3, 3))
    (9): ReLU()
    (10): Conv2d(256, 128, kernel_size=(7, 7), stride=(1, 1), padd

In [11]:
test_letter = x_train[:32, :, :26].cuda()
test_image = x_train[:32, :, 26:].reshape(-1, 1, 28, 28).cuda()

In [12]:
print(test_letter.shape)
print(test_image.shape)

torch.Size([32, 1, 26])
torch.Size([32, 1, 28, 28])


In [13]:
model(test_letter, test_image)

tensor([[0.1212, 0.0993, 0.0869, 0.0981, 0.0895, 0.1167, 0.1026, 0.0923, 0.1055,
         0.0881],
        [0.1212, 0.0993, 0.0869, 0.0981, 0.0895, 0.1167, 0.1026, 0.0922, 0.1054,
         0.0881],
        [0.1212, 0.0993, 0.0869, 0.0981, 0.0895, 0.1167, 0.1026, 0.0923, 0.1054,
         0.0881],
        [0.1212, 0.0993, 0.0869, 0.0981, 0.0895, 0.1167, 0.1026, 0.0922, 0.1054,
         0.0881],
        [0.1212, 0.0993, 0.0869, 0.0981, 0.0895, 0.1167, 0.1026, 0.0922, 0.1054,
         0.0881],
        [0.1212, 0.0993, 0.0869, 0.0981, 0.0895, 0.1167, 0.1026, 0.0922, 0.1054,
         0.0881],
        [0.1212, 0.0993, 0.0869, 0.0981, 0.0895, 0.1167, 0.1026, 0.0922, 0.1054,
         0.0881],
        [0.1212, 0.0993, 0.0869, 0.0981, 0.0895, 0.1167, 0.1026, 0.0922, 0.1054,
         0.0881],
        [0.1212, 0.0993, 0.0869, 0.0981, 0.0895, 0.1167, 0.1026, 0.0922, 0.1055,
         0.0881],
        [0.1212, 0.0993, 0.0869, 0.0981, 0.0895, 0.1167, 0.1026, 0.0922, 0.1054,
         0.0881],
        [0

In [14]:
from torch.optim import Adam

optimizer = Adam(
    model.parameters(),
    lr=2e-5,
    eps=1e-8,
)

In [15]:
epochs = 150
seed_val = 42
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [16]:
device = torch.device('cuda')
print(device)

cuda


In [17]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [18]:
# gradient 초기화
model.zero_grad()

history = defaultdict(list)
for epoch_i in range(0, epochs):
    
    total_loss = 0
    
    # train 모드로 변경
    model.train()
    
    # dataloader에서 batch size만큼 반복해서 가져옴
    for step, batch in enumerate(train_dataloader):
        
        # batch를 GPU에 적용
        batch = tuple(t.to(device) for t in batch)
        
        # batch에서 데이터 추출
        letter, image, label = batch
        
        # Forward Propagation 수행
        outputs = model(letter, image, label)
        
        loss = outputs[1]
        
        total_loss += loss.item()
        
        # Backward Propagation 수행
        loss.backward()
        history["train_loss"].append(loss.item())
        
        # 정확도 계산
        logits = outputs[0].detach().cpu().numpy()
        label = label.to("cpu").numpy()
        tmp_train_accuracy = flat_accuracy(logits, label)
        history["train_acc"].append(tmp_train_accuracy)
        
        # Gradient Cleeping
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        # gradient를 통해 weight update
        optimizer.step()
        
        # gradient 초기화
        model.zero_grad()
        
    # average loss
    avg_train_loss = total_loss / len(train_dataloader)
    
    # ========================================
    #               Validation
    # ========================================
    
    t0 = time.time()
    
    # eval 모드로 변경
    model.eval()
    
    # 변수 초기화
    eval_loss, eval_accuracy, nb_eval_steps, nb_eval_examples = 0, 0, 0, 0
    
    # dataloader에서 batch만큼 반복해서 가져옴
    for batch in valid_dataloader:
        
        # batch를 GPU에 적용
        batch = tuple(t.to(device) for t in batch)
        
        # batch에서 데이터 추출
        letter, image, label = batch
        
        # gradient 계산 안함
        with torch.no_grad():
            # Forward Propagation 수행
            outputs = model(letter, image, label)
        
        logits = outputs[0]
        history["eval_loss"].append(outputs[1].item())
        
        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label = label.to("cpu").numpy()
        
        # 출력 logit과 label을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label)
        history["eval_acc"].append(tmp_eval_accuracy)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1
        
    s = f"\r[Epoch {epoch_i+1}/{epochs}]"
    s += f" Avg Training Loss: {avg_train_loss: .2f}"
    s += " Valid Acc: {0:.2f}".format(eval_accuracy / nb_eval_steps)
    print(s, end="")
    
print("")
print("Training complete")

[Epoch 150/150] Avg Training Loss:  1.55 Valid Acc: 0.61
Training complete


In [19]:
torch.save(model.state_dict(), "./model/emnist_model3.pt")

In [20]:
model = customCNN()
model.load_state_dict(torch.load("./model/emnist_model3.pt"))
model.eval()
model.cuda()

customCNN(
  (conv1): Sequential(
    (0): Conv1d(1, 16, kernel_size=(3,), stride=(1,), padding=(1,))
    (1): ReLU()
    (2): Conv1d(16, 64, kernel_size=(4,), stride=(1,), padding=(1,))
    (3): ReLU()
    (4): Conv1d(64, 128, kernel_size=(5,), stride=(1,), padding=(2,))
    (5): ReLU()
    (6): Conv1d(128, 64, kernel_size=(4,), stride=(1,), padding=(2,))
    (7): ReLU()
    (8): Conv1d(64, 16, kernel_size=(3,), stride=(1,))
    (9): ReLU()
  )
  (conv2): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): Conv2d(32, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (3): ReLU()
    (4): Conv2d(128, 256, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (5): ReLU()
    (6): Conv2d(256, 512, kernel_size=(9, 9), stride=(1, 1), padding=(3, 3))
    (7): ReLU()
    (8): Conv2d(512, 256, kernel_size=(9, 9), stride=(1, 1), padding=(3, 3))
    (9): ReLU()
    (10): Conv2d(256, 128, kernel_size=(7, 7), stride=(1, 1), padd

In [21]:
x_test = np.concatenate(
    [
        pd.get_dummies(test["letter"]).values.reshape(-1, 1, 26),
        (test[[str(i) for i in range(784)]] / 255.).values.reshape(-1, 1, 784)
    ],
    axis=2
)
x_test = torch.Tensor(x_test)

x1 = x_test[:, :, :26].cuda()
x2 = x_test[:, :, 26:].reshape(-1, 1, 28, 28).cuda()

In [22]:
test_data = TensorDataset(x1, x2)
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=32)

In [23]:
y_pred = []
for batch in test_dataloader:
    input1, input2 = batch
    with torch.no_grad():
        outputs = model(input1, input2)
    y_pred.append(torch.argmax(outputs, dim=1))

In [24]:
submission["digit"] = torch.cat(y_pred).detach().cpu().numpy()
submission.to_csv("./result/submission3.csv", index=False)